In [0]:
dbutils.fs.mount(
source = 'wasbs://energy-climate-data@energyclimatedata.blob.core.windows.net', # contrainer@storageacc
mount_point = '/mnt/energyclimatenexus',
extra_configs = {'fs.azure.account.key.energyclimatedata.blob.core.windows.net':'jcLtlXk09OHdNjb1AdkjmC3rgmnD5dM8oukeYudNceBdzcgQ1UFL0I7zEJ+ok4mykJBvmWcM7TCH+ASt1ZedzQ=='})


---------------------------------------------------------------------------
ExecutionError                            Traceback (most recent call last)
File <command-294508866870502>:1
----> 1 dbutils.fs.mount(
      2 source = 'wasbs://energy-climate-data@energyclimatedata.blob.core.windows.net', # contrainer@storageacc
      3 mount_point = '/mnt/energyclimatenexus',
      4 extra_configs = {'fs.azure.account.key.energyclimatedata.blob.core.windows.net':'jcLtlXk09OHdNjb1AdkjmC3rgmnD5dM8oukeYudNceBdzcgQ1UFL0I7zEJ+ok4mykJBvmWcM7TCH+ASt1ZedzQ=='})

File /databricks/python_shell/dbruntime/dbutils.py:362, in DBUtils.FSHandler.prettify_exception_message.<locals>.f_with_exception_handling(*args, **kwargs)
    360 exc.__context__ = None
    361 exc.__cause__ = None
--> 362 raise exc

ExecutionError: An error occurred while calling o375.mount.
: java.rmi.RemoteException: java.lang.IllegalArgumentException: requirement failed: Directory already mounted: /mnt/energyclimatenexus; nested exceptio

In [0]:
dbutils.fs.ls('/mnt/energyclimatenexus/Raw_data')
#dbutils.fs.unmount('/mnt/energyclimatenexus')

Out[1]: [FileInfo(path='dbfs:/mnt/energyclimatenexus/Raw_data/CO2_emissions_data.csv', name='CO2_emissions_data.csv', size=3418932, modificationTime=1696016069000),
 FileInfo(path='dbfs:/mnt/energyclimatenexus/Raw_data/climate_data.csv', name='climate_data.csv', size=135844, modificationTime=1696016052000),
 FileInfo(path='dbfs:/mnt/energyclimatenexus/Raw_data/non_ren_energy.csv', name='non_ren_energy.csv', size=11017751, modificationTime=1696016038000),
 FileInfo(path='dbfs:/mnt/energyclimatenexus/Raw_data/ren_energy.csv', name='ren_energy.csv', size=7763966, modificationTime=1696016014000)]

In [0]:
spark

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.3.2 
 Master 
 local[8] 
 AppName 
 Databricks Shell

In [0]:
df_nre = spark.read.csv("dbfs:/mnt/energyclimatenexus/Raw_data/non_ren_energy.csv", header=True, inferSchema=True)
df_nre.count()

Out[4]: 93130

In [0]:
df_nre.show()

+------+--------+--------------------+-------+--------------------+-----+--------------------+
|period|seriesId|   seriesDescription|stateId|    stateDescription|value|                unit|
+------+--------+--------------------+-------+--------------------+-----+--------------------+
|  2021|   PACCD|All petroleum pro...|     AK|              Alaska|19.09|Dollars per milli...|
|  2021|   PACCD|All petroleum pro...|     AL|             Alabama|19.46|Dollars per milli...|
|  2021|   PACCD|All petroleum pro...|     AR|            Arkansas|19.47|Dollars per milli...|
|  2021|   PACCD|All petroleum pro...|     AZ|             Arizona|24.07|Dollars per milli...|
|  2021|   PACCD|All petroleum pro...|     CA|          California|27.58|Dollars per milli...|
|  2021|   PACCD|All petroleum pro...|     CO|            Colorado| 22.3|Dollars per milli...|
|  2021|   PACCD|All petroleum pro...|     CT|         Connecticut|20.86|Dollars per milli...|
|  2021|   PACCD|All petroleum pro...|     DE|    

In [0]:
df_nre.describe().show()

+-------+------------------+--------+--------------------+-------+----------------+-----------------+-------------------+
|summary|            period|seriesId|   seriesDescription|stateId|stateDescription|            value|               unit|
+-------+------------------+--------+--------------------+-------+----------------+-----------------+-------------------+
|  count|             93130|   93130|               93130|  93130|           93130|            93130|              93130|
|   mean|2010.5310318909053|    null|                null|   null|            null|62526.55142628601|               null|
| stddev| 6.321463575526689|    null|                null|   null|            null|267997.6422437851|               null|
|    min|              2000|   CLACB|All petroleum pro...|     AK|         Alabama|              0.0|            Barrels|
|    max|              2021|   PATPP|Nuclear fuel tota...|     WY|         Wyoming|        8087435.0|Thousand short tons|
+-------+---------------

In [0]:
df_nre.printSchema()

root
 |-- period: integer (nullable = true)
 |-- seriesId: string (nullable = true)
 |-- seriesDescription: string (nullable = true)
 |-- stateId: string (nullable = true)
 |-- stateDescription: string (nullable = true)
 |-- value: double (nullable = true)
 |-- unit: string (nullable = true)



In [0]:
#Check Nulls in a DF
from pyspark.sql.functions import col,isnan, when, count
df_nre.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_nre.columns]
   ).show()
#df=spark.sql("select * from nre where coalesce(period,seriesId,seriesDescription,stateId,stateDescription,value,unit) is null")
#df.count()

+------+--------+-----------------+-------+----------------+-----+----+
|period|seriesId|seriesDescription|stateId|stateDescription|value|unit|
+------+--------+-----------------+-------+----------------+-----+----+
|     0|       0|                0|      0|               0|    0|   0|
+------+--------+-----------------+-------+----------------+-----+----+



In [0]:
#Check the duplicates
df_nre.createOrReplaceTempView("nre")
spark.sql("select * from nre group by all having count(*)>1").count()


Out[9]: 0

In [0]:
#Drop Duplicates
df_nre=df_nre.dropDuplicates()
df_nre.count()

Out[10]: 92004

In [0]:

# List of values to filter on
values_to_filter = ['Billion Btu', 'Dollars per million Btu', 'Million Btu','Million dollars','Million kilowatthours']

# Applying the filter
filtered_df_nre = df_nre.filter(col('unit').isin(values_to_filter))

# Show the filtered DataFrame
filtered_df_nre.select('unit').distinct().show()
filtered_df_nre.count()

+--------------------+
|                unit|
+--------------------+
|         Billion Btu|
|Dollars per milli...|
|Million kilowatth...|
|     Million dollars|
|         Million Btu|
+--------------------+

Out[13]: 68442

In [0]:
conditions_col1 = [
    (col("seriesDescription").contains("commercial"), "Commercial"),
    (col("seriesDescription").contains("residential"),"Residential"),
    (col("seriesDescription").contains("industrial"), "Industrial"),
    (col("seriesDescription").contains("transportation"), "Transportation"),
    (col("seriesDescription").contains("electric power"), "Electric Power"),
    (col("seriesDescription").contains("total"), "All Sectors"),
    (col("seriesDescription").contains("all sectors"), "All Sectors")
    # Add more conditions for col1 as needed
]

conditions_col2 = [
    (col("seriesDescription").contains("petrol"), "All Petroleum Products"),
    (col("seriesDescription").contains("Natural"), "Natural Gas"),
    (col("seriesDescription").contains("Coal"), "Coal"),
    (col("seriesDescription").contains("Nuclear"), "Nuclear")
    # Add more conditions for col2 as needed
]

conditions_col3 = [
    (col("seriesDescription").contains("price"), "Average Price"),
    (col("seriesDescription").contains("consumption per capita"), "Consumption per capita"),
    (col("seriesDescription").contains("consum"), "Consumption"),
    (col("seriesDescription").contains("expenditure"), "Expenditure"),
    (col("seriesDescription").contains("production"), "Production"),
    (col("seriesDescription").contains("generation"), "Generation"),
    # Add more conditions for col2 as needed
]

conditions_col4 = [
    (col("stateDescription").isin("Connecticut", "Maine", "Massachusetts", "New Hampshire", "New Jersey",
                        "New York", "Pennsylvania", "Rhode Island", "Vermont"), "Northeast"),
    (col("stateDescription").isin("Illinois", "Indiana", "Iowa", "Kansas", "Michigan", "Minnesota",
                        "Missouri", "Nebraska", "North Dakota", "Ohio", "South Dakota", "Wisconsin"), "Midwest"),
    (col("stateDescription").isin("Alabama", "Arkansas", "Delaware", "District of Columbia", "Florida", "Georgia",
                        "Kentucky", "Louisiana", "Maryland", "Mississippi", "North Carolina", "Oklahoma",
                        "South Carolina", "Tennessee", "Texas", "Virginia", "West Virginia"), "South"),
    (col("stateDescription").isin("Alaska", "Arizona", "California", "Colorado", "Hawaii", "Idaho", "Montana",
                        "Nevada", "New Mexico", "Oregon", "Utah", "Washington", "Wyoming"), "West")
]

result_df_nre = filtered_df_nre \
    .withColumn("sector", 
                when(conditions_col1[0][0], conditions_col1[0][1])
                .when(conditions_col1[1][0], conditions_col1[1][1])
                .when(conditions_col1[2][0], conditions_col1[2][1])
                .when(conditions_col1[3][0], conditions_col1[3][1])
                .when(conditions_col1[4][0], conditions_col1[4][1])
                .when(conditions_col1[5][0], conditions_col1[5][1])
                .when(conditions_col1[6][0], conditions_col1[6][1])
                .otherwise("All Sectors")) \
    .withColumn("energy_source", 
                when(conditions_col2[0][0], conditions_col2[0][1])
                .when(conditions_col2[1][0], conditions_col2[1][1])
                .when(conditions_col2[2][0], conditions_col2[2][1])
                .when(conditions_col2[3][0], conditions_col2[3][1])
                .otherwise("na")) \
    .withColumn("type", 
                when(conditions_col3[0][0], conditions_col3[0][1])
                .when(conditions_col3[1][0], conditions_col3[1][1])
                .when(conditions_col3[2][0], conditions_col3[2][1])
                .when(conditions_col3[3][0], conditions_col3[3][1])
                .when(conditions_col3[4][0], conditions_col3[4][1])
                .when(conditions_col3[5][0], conditions_col3[5][1])
                .otherwise("na")) \
    .withColumn("regions", 
                when(conditions_col4[0][0], conditions_col4[0][1])
                .when(conditions_col4[1][0], conditions_col4[1][1])
                .when(conditions_col4[2][0], conditions_col4[2][1])
                .when(conditions_col4[3][0], conditions_col4[3][1])
                .otherwise("Other"))

result_df_nre.show()
result_df_nre.count()

+------+--------+--------------------+-------+--------------------+-----+--------------------+----------+--------------------+-------------+---------+
|period|seriesId|   seriesDescription|stateId|    stateDescription|value|                unit|    sector|       energy_source|         type|  regions|
+------+--------+--------------------+-------+--------------------+-----+--------------------+----------+--------------------+-------------+---------+
|  2021|   PACCD|All petroleum pro...|     LA|           Louisiana|19.58|Dollars per milli...|Commercial|All Petroleum Pro...|Average Price|    South|
|  2021|   PACCD|All petroleum pro...|     AZ|             Arizona|24.07|Dollars per milli...|Commercial|All Petroleum Pro...|Average Price|     West|
|  2021|   PACCD|All petroleum pro...|     ID|               Idaho| 22.5|Dollars per milli...|Commercial|All Petroleum Pro...|Average Price|     West|
|  2021|   PACCD|All petroleum pro...|     ME|               Maine|19.85|Dollars per milli...|

In [0]:
from pyspark.sql.functions import col,isnan, when, count
result_df_nre.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in result_df_nre.columns]
   ).show()

In [0]:
distinct_values = result_df_nre.select('sector', 'energy_source', 'type').distinct().orderBy('sector','energy_source','type')
distinct_values.show(200)

In [0]:
distinct_values = result_df_nre.select( 'type','unit').distinct().orderBy('type','unit')
distinct_values.show(200)

+--------------------+--------------------+
|                type|                unit|
+--------------------+--------------------+
|       Average Price|Dollars per milli...|
|         Consumption|         Billion Btu|
|Consumption per c...|         Million Btu|
|         Expenditure|     Million dollars|
|          Generation|Million kilowatth...|
|          Production|         Billion Btu|
+--------------------+--------------------+



In [0]:
result_df_nre.coalesce(1).write.mode("overwrite").option("header",'true').csv("/mnt/energyclimatenexus/Transformed_data/nre")

In [0]:
df_re = spark.read.csv("dbfs:/mnt/energyclimatenexus/Raw_data/ren_energy.csv", header=True, inferSchema=True)
df_re.count()

Out[16]: 70482

In [0]:
df_re.show()

+------+--------+-------------------+-------+--------------------+-------+----------------+
|period|seriesId|  seriesDescription|stateId|    stateDescription|  value|            unit|
+------+--------+-------------------+-------+--------------------+-------+----------------+
|  2021|   BFPRP|Biofuels production|     AK|              Alaska|    0.0|Thousand barrels|
|  2021|   BFPRB|Biofuels production|     AK|              Alaska|    0.0|     Billion Btu|
|  2021|   BFPRP|Biofuels production|     AL|             Alabama|  297.0|Thousand barrels|
|  2021|   BFPRB|Biofuels production|     AL|             Alabama| 1615.0|     Billion Btu|
|  2021|   BFPRB|Biofuels production|     AR|            Arkansas| 6058.0|     Billion Btu|
|  2021|   BFPRP|Biofuels production|     AR|            Arkansas| 1115.0|Thousand barrels|
|  2021|   BFPRP|Biofuels production|     AZ|             Arizona|    0.0|Thousand barrels|
|  2021|   BFPRB|Biofuels production|     AZ|             Arizona|    0.0|     B

In [0]:
df_re.describe().show()

+-------+------------------+--------+--------------------+-------+----------------+-----------------+----------------+
|summary|            period|seriesId|   seriesDescription|stateId|stateDescription|            value|            unit|
+-------+------------------+--------+--------------------+-------+----------------+-----------------+----------------+
|  count|             70482|   70482|               70482|  70482|           70482|            70482|           70482|
|   mean|2010.4965097471695|    null|                null|   null|            null| 9542.65666992989|            null|
| stddev|  6.35039493294208|    null|                null|   null|            null|35569.63266047802|            null|
|    min|              2000|   BFPRB| Biofuels production|     AK|         Alabama|             -2.0|     Billion Btu|
|    max|              2021|   WYTCP|Wood price in the...|     WY|         Wyoming|         892101.0|Thousand barrels|
+-------+------------------+--------+-----------

In [0]:
df_re.printSchema()

root
 |-- period: integer (nullable = true)
 |-- seriesId: string (nullable = true)
 |-- seriesDescription: string (nullable = true)
 |-- stateId: string (nullable = true)
 |-- stateDescription: string (nullable = true)
 |-- value: double (nullable = true)
 |-- unit: string (nullable = true)



In [0]:
#Check Nulls in a DF
from pyspark.sql.functions import col,isnan, when, count
df_re.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_re.columns]
   ).show()

+------+--------+-----------------+-------+----------------+-----+----+
|period|seriesId|seriesDescription|stateId|stateDescription|value|unit|
+------+--------+-----------------+-------+----------------+-----+----+
|     0|       0|                0|      0|               0|    0|   0|
+------+--------+-----------------+-------+----------------+-----+----+



In [0]:
#Check the duplicates
df_re.createOrReplaceTempView("re")
spark.sql("select * from re group by all having count(*)>1").count()


Out[18]: 52

In [0]:
#Drop Duplicates
df_re=df_re.dropDuplicates()
df_re.count()

Out[19]: 70430

In [0]:
# List of values to filter on
values_to_filter = ['Billion Btu', 'Dollars per million Btu', 'Million Btu','Million dollars','Million kilowatthours']
values_to_remove = ['Wood and waste energy consumed in the commercial sector at a cost','Wood and waste energy consumed in the commercial sector at no cost','Wood and waste energy consumed in the industrial sector at a cost','Wood and waste energy consumed in the industrial sector at no cost','Wood energy consumed in the residential sector at a cost','Wood energy consumed in the residential sector at no cost']

# Applying the filter
df_re=df_re.filter(~col('seriesDescription').isin(values_to_remove))
filtered_df_re = df_re.filter(col('unit').isin(values_to_filter))

# Show the filtered DataFrame
filtered_df_re.select('unit').distinct().show()
filtered_df_re.count()

+--------------------+
|                unit|
+--------------------+
|         Billion Btu|
|Dollars per milli...|
|Million kilowatth...|
|     Million dollars|
+--------------------+

Out[20]: 62576

In [0]:
conditions_col1 = [
    (col("seriesDescription").contains("commercial"), "Commercial"),
    (col("seriesDescription").contains("residential"),"Residential"),
    (col("seriesDescription").contains("industrial"), "Industrial"),
    (col("seriesDescription").contains("transportation"), "Transportation"),
    (col("seriesDescription").contains("electric power"), "Electric Power"),
    (col("seriesDescription").contains("total"), "All Sectors"),
    (col("seriesDescription").contains("all sectors"), "All Sectors")
   
]

conditions_col2 = [
    (col("seriesDescription").contains("Biofuels"), "Biofuels"),
    (col("seriesDescription").contains("Hydro"), "Hydro"),
    (col("seriesDescription").contains("Geothermal"), "Geothermal"),
    (col("seriesDescription").contains("Solar"), "Solar"),
    (col("seriesDescription").contains("Wind"), "Wind"),
    (col("seriesDescription").contains("Wood and waste"), "Wood and waste"),
    (col("seriesDescription").contains("Wood"), "Wood")
    
]

conditions_col3 = [
    (col("seriesDescription").contains("price"), "Average Price"),
    (col("seriesDescription").contains("consumption per capita"), "Consumption per capita"),
    (col("seriesDescription").contains("consum"), "Consumption"),
    (col("seriesDescription").contains("expenditure"), "Expenditure"),
    (col("seriesDescription").contains("production"), "Production"),
    (col("seriesDescription").contains("generation"), "Generation"),
    
]

conditions_col4 = [
    (col("stateDescription").isin("Connecticut", "Maine", "Massachusetts", "New Hampshire", "New Jersey",
                        "New York", "Pennsylvania", "Rhode Island", "Vermont"), "Northeast"),
    (col("stateDescription").isin("Illinois", "Indiana", "Iowa", "Kansas", "Michigan", "Minnesota",
                        "Missouri", "Nebraska", "North Dakota", "Ohio", "South Dakota", "Wisconsin"), "Midwest"),
    (col("stateDescription").isin("Alabama", "Arkansas", "Delaware", "District of Columbia", "Florida", "Georgia",
                        "Kentucky", "Louisiana", "Maryland", "Mississippi", "North Carolina", "Oklahoma",
                        "South Carolina", "Tennessee", "Texas", "Virginia", "West Virginia"), "South"),
    (col("stateDescription").isin("Alaska", "Arizona", "California", "Colorado", "Hawaii", "Idaho", "Montana",
                        "Nevada", "New Mexico", "Oregon", "Utah", "Washington", "Wyoming"), "West")
]

result_df_re = filtered_df_re \
    .withColumn("sector", 
                when(conditions_col1[0][0], conditions_col1[0][1])
                .when(conditions_col1[1][0], conditions_col1[1][1])
                .when(conditions_col1[2][0], conditions_col1[2][1])
                .when(conditions_col1[3][0], conditions_col1[3][1])
                .when(conditions_col1[4][0], conditions_col1[4][1])
                .when(conditions_col1[5][0], conditions_col1[5][1])
                .when(conditions_col1[6][0], conditions_col1[6][1])
                .otherwise("All Sectors")) \
    .withColumn("energy_source", 
                when(conditions_col2[0][0], conditions_col2[0][1])
                .when(conditions_col2[1][0], conditions_col2[1][1])
                .when(conditions_col2[2][0], conditions_col2[2][1])
                .when(conditions_col2[3][0], conditions_col2[3][1])
                .when(conditions_col2[4][0], conditions_col2[4][1])
                .when(conditions_col2[5][0], conditions_col2[5][1])
                .when(conditions_col2[6][0], conditions_col2[6][1])
                .otherwise("na")) \
    .withColumn("type", 
                when(conditions_col3[0][0], conditions_col3[0][1])
                .when(conditions_col3[1][0], conditions_col3[1][1])
                .when(conditions_col3[2][0], conditions_col3[2][1])
                .when(conditions_col3[3][0], conditions_col3[3][1])
                .when(conditions_col3[4][0], conditions_col3[4][1])
                .when(conditions_col3[5][0], conditions_col3[5][1])
                .otherwise("na")) \
    .withColumn("regions", 
                when(conditions_col4[0][0], conditions_col4[0][1])
                .when(conditions_col4[1][0], conditions_col4[1][1])
                .when(conditions_col4[2][0], conditions_col4[2][1])
                .when(conditions_col4[3][0], conditions_col4[3][1])
                .otherwise("Other"))

result_df_re.show()
result_df_re.count()

+------+--------+-------------------+-------+--------------------+--------+-----------+-----------+-------------+----------+---------+
|period|seriesId|  seriesDescription|stateId|    stateDescription|   value|       unit|     sector|energy_source|      type|  regions|
+------+--------+-------------------+-------+--------------------+--------+-----------+-----------+-------------+----------+---------+
|  2021|   BFPRB|Biofuels production|     HI|              Hawaii|   711.0|Billion Btu|All Sectors|     Biofuels|Production|     West|
|  2021|   BFPRB|Biofuels production|     KS|              Kansas| 80295.0|Billion Btu|All Sectors|     Biofuels|Production|  Midwest|
|  2021|   BFPRB|Biofuels production|     CA|          California| 36702.0|Billion Btu|All Sectors|     Biofuels|Production|     West|
|  2021|   BFPRB|Biofuels production|     IA|                Iowa|597174.0|Billion Btu|All Sectors|     Biofuels|Production|  Midwest|
|  2021|   BFPRB|Biofuels production|     MD|          

In [0]:
from pyspark.sql.functions import col,isnan, when, count
result_df_re.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in result_df_re.columns]
   ).show()

In [0]:
result_df_re.select('sector', 'energy_source', 'type').distinct().orderBy('sector','energy_source','type').show(300)

In [0]:
distinct_values = result_df_re.select( 'type','unit').distinct().orderBy('type','unit')
distinct_values.show(200)

+-------------+--------------------+
|         type|                unit|
+-------------+--------------------+
|Average Price|Dollars per milli...|
|  Consumption|         Billion Btu|
|  Expenditure|     Million dollars|
|   Generation|Million kilowatth...|
|   Production|         Billion Btu|
+-------------+--------------------+



In [0]:
result_df_re.filter((col('energy_source') == 'Wood') &  (col('sector')=='Residential') & (col('stateId')=='AK')).orderBy('period','type').show()


In [0]:
result_df_re.coalesce(1).write.mode("overwrite").option("header",'true').csv("/mnt/energyclimatenexus/Transformed_data/re")

In [0]:
df_cl = spark.read.csv("dbfs:/mnt/energyclimatenexus/Raw_data/climate_data.csv", header=True, inferSchema=True)
df_cl.count()

Out[58]: 1127

In [0]:
df_cl.show()

+------+-------+----------+------------+----------------+--------+----------------+--------+----------------+-------------+---------------------+---------------------------------+-----------+----+-----------+----+-----------+
|  Year|Stateid|State_name|Average Temp|Avg_Temp_Anamoly|Max temp|Max temp Anamoly|Min Temp|Min Temp Anamoly|Precipitation|Precipitation Anamoly|Drought Severity (Palmar Z Index)|DSI Anamoly| CDD|CDD Anamoly| HDD|HDD Anamoly|
+------+-------+----------+------------+----------------+--------+----------------+--------+----------------+-------------+---------------------+---------------------------------+-----------+----+-----------+----+-----------+
|200012|     AL|   Alabama|        63.6|             0.5|    76.0|             1.1|    51.2|             0.0|        42.49|               -12.76|                            -1.42|      -1.38|2052|        167|2802|        -22|
|200112|     AL|   Alabama|        63.0|            -0.1|    74.6|            -0.3|    51.4|    

In [0]:
df_cl.printSchema()
df_cl.describe().show()

root
 |-- Year: integer (nullable = true)
 |-- Stateid: string (nullable = true)
 |-- State_name: string (nullable = true)
 |-- Average Temp: double (nullable = true)
 |-- Avg_Temp_Anamoly: double (nullable = true)
 |-- Max temp: double (nullable = true)
 |-- Max temp Anamoly: double (nullable = true)
 |-- Min Temp: double (nullable = true)
 |-- Min Temp Anamoly: double (nullable = true)
 |-- Precipitation: double (nullable = true)
 |-- Precipitation Anamoly: double (nullable = true)
 |-- Drought Severity (Palmar Z Index): double (nullable = true)
 |-- DSI Anamoly: double (nullable = true)
 |-- CDD: integer (nullable = true)
 |-- CDD Anamoly: integer (nullable = true)
 |-- HDD: integer (nullable = true)
 |-- HDD Anamoly: integer (nullable = true)

+-------+-----------------+-------+----------+-----------------+------------------+-----------------+------------------+-----------------+------------------+------------------+---------------------+---------------------------------+----------

In [0]:
from pyspark.sql.functions import col,isnan, when, count
df_cl = df_cl.withColumn("year", col("Year").substr(1, 4).cast("int"))
df_cl.show()

+----+-------+----------+------------+----------------+--------+----------------+--------+----------------+-------------+---------------------+---------------------------------+-----------+----+-----------+----+-----------+
|year|Stateid|State_name|Average Temp|Avg_Temp_Anamoly|Max temp|Max temp Anamoly|Min Temp|Min Temp Anamoly|Precipitation|Precipitation Anamoly|Drought Severity (Palmar Z Index)|DSI Anamoly| CDD|CDD Anamoly| HDD|HDD Anamoly|
+----+-------+----------+------------+----------------+--------+----------------+--------+----------------+-------------+---------------------+---------------------------------+-----------+----+-----------+----+-----------+
|2000|     AL|   Alabama|        63.6|             0.5|    76.0|             1.1|    51.2|             0.0|        42.49|               -12.76|                            -1.42|      -1.38|2052|        167|2802|        -22|
|2001|     AL|   Alabama|        63.0|            -0.1|    74.6|            -0.3|    51.4|             0

In [0]:
#Check Nulls in a DF
from pyspark.sql.functions import col,isnan, when, count
df_cl.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_cl.columns]
   ).show()
#Check the duplicates
df_cl.createOrReplaceTempView("climate")
spark.sql("select * from climate group by all having count(*)>1").count()

+----+-------+----------+------------+----------------+--------+----------------+--------+----------------+-------------+---------------------+---------------------------------+-----------+---+-----------+---+-----------+
|year|Stateid|State_name|Average Temp|Avg_Temp_Anamoly|Max temp|Max temp Anamoly|Min Temp|Min Temp Anamoly|Precipitation|Precipitation Anamoly|Drought Severity (Palmar Z Index)|DSI Anamoly|CDD|CDD Anamoly|HDD|HDD Anamoly|
+----+-------+----------+------------+----------------+--------+----------------+--------+----------------+-------------+---------------------+---------------------------------+-----------+---+-----------+---+-----------+
|   0|      0|         0|           0|               0|       0|               0|       0|               0|            0|                    0|                                0|          0|  0|          0|  0|          0|
+----+-------+----------+------------+----------------+--------+----------------+--------+----------------+-----

In [0]:
conditions_col1 = [
    (col("State_name").isin("Connecticut", "Maine", "Massachusetts", "New Hampshire", "New Jersey",
                        "New York", "Pennsylvania", "Rhode Island", "Vermont"), "Northeast"),
    (col("State_name").isin("Illinois", "Indiana", "Iowa", "Kansas", "Michigan", "Minnesota",
                        "Missouri", "Nebraska", "North Dakota", "Ohio", "South Dakota", "Wisconsin"), "Midwest"),
    (col("State_name").isin("Alabama", "Arkansas", "Delaware", "District of Columbia", "Florida", "Georgia",
                        "Kentucky", "Louisiana", "Maryland", "Mississippi", "North Carolina", "Oklahoma",
                        "South Carolina", "Tennessee", "Texas", "Virginia", "West Virginia"), "South"),
    (col("State_name").isin("Alaska", "Arizona", "California", "Colorado", "Hawaii", "Idaho", "Montana",
                        "Nevada", "New Mexico", "Oregon", "Utah", "Washington", "Wyoming"), "West")
]

result_df_cl = df_cl \
   .withColumn("regions", 
                when(conditions_col1[0][0], conditions_col1[0][1])
                .when(conditions_col1[1][0], conditions_col1[1][1])
                .when(conditions_col1[2][0], conditions_col1[2][1])
                .when(conditions_col1[3][0], conditions_col1[3][1])
                .otherwise("Other")) 

result_df_cl.show()
result_df_cl.count()

+----+-------+----------+------------+----------------+--------+----------------+--------+----------------+-------------+---------------------+---------------------------------+-----------+----+-----------+----+-----------+-------+
|year|Stateid|State_name|Average Temp|Avg_Temp_Anamoly|Max temp|Max temp Anamoly|Min Temp|Min Temp Anamoly|Precipitation|Precipitation Anamoly|Drought Severity (Palmar Z Index)|DSI Anamoly| CDD|CDD Anamoly| HDD|HDD Anamoly|regions|
+----+-------+----------+------------+----------------+--------+----------------+--------+----------------+-------------+---------------------+---------------------------------+-----------+----+-----------+----+-----------+-------+
|2000|     AL|   Alabama|        63.6|             0.5|    76.0|             1.1|    51.2|             0.0|        42.49|               -12.76|                            -1.42|      -1.38|2052|        167|2802|        -22|  South|
|2001|     AL|   Alabama|        63.0|            -0.1|    74.6|        

In [0]:
result_df_cl.coalesce(1).write.mode("overwrite").option("header",'true').csv("/mnt/energyclimatenexus/Transformed_data/climate_data")

In [0]:
df_co = spark.read.csv("dbfs:/mnt/energyclimatenexus/Raw_data/CO2_emissions_data.csv", header=True, inferSchema=True)
df_co.count()

Out[62]: 26928

In [0]:
df_co.show()

+------+--------+--------------------+------+-----------+-------+----------+---------+--------------------+
|period|sectorId|         sector-name|fuelId|  fuel-name|stateId|state-name|    value|         value-units|
+------+--------+--------------------+------+-----------+-------+----------+---------+--------------------+
|  2021|      RC|Residential carbo...|    CO|       Coal|     AK|    Alaska|      0.0|million metric to...|
|  2021|      TT|Total carbon diox...|    NG|Natural Gas|     AK|    Alaska|19.623623|million metric to...|
|  2021|      RC|Residential carbo...|    PE|  Petroleum|     AK|    Alaska| 0.616132|million metric to...|
|  2021|      RC|Residential carbo...|    NG|Natural Gas|     AK|    Alaska| 1.113967|million metric to...|
|  2021|      RC|Residential carbo...|    TO|  All Fuels|     AK|    Alaska| 1.730099|million metric to...|
|  2021|      CC|Commercial carbon...|    CO|       Coal|     AK|    Alaska| 0.706643|million metric to...|
|  2021|      CC|Commercial 

In [0]:
df_co = df_co.withColumnRenamed("state-name", "State_name")
df_co = df_co.withColumnRenamed("sector-name", "Sector")
df_co = df_co.withColumnRenamed("fuel-name", "Fuel")
df_co = df_co.withColumnRenamed("value-units", "unit")

In [0]:
df_co.printSchema()
df_co.describe().show()

root
 |-- period: integer (nullable = true)
 |-- sectorId: string (nullable = true)
 |-- Sector: string (nullable = true)
 |-- fuelId: string (nullable = true)
 |-- Fuel: string (nullable = true)
 |-- stateId: string (nullable = true)
 |-- State_name: string (nullable = true)
 |-- value: double (nullable = true)
 |-- unit: string (nullable = true)

+-------+-----------------+--------+--------------------+------+---------+-------+----------+-----------------+--------------------+
|summary|           period|sectorId|              Sector|fuelId|     Fuel|stateId|State_name|            value|                unit|
+-------+-----------------+--------+--------------------+------+---------+-------+----------+-----------------+--------------------+
|  count|            26928|   26928|               26928| 26928|    26928|  26928|     26928|            26928|               26928|
|   mean|           2010.5|    null|                null|  null|     null|   null|      null|17.96557888766334|      

In [0]:
#Check Nulls in a DF
from pyspark.sql.functions import col,isnan, when, count
df_co.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in df_co.columns]
   ).show()
#Check the duplicates
df_co.createOrReplaceTempView("co2")
spark.sql("select * from co2 group by all having count(*)>1").count()

+------+--------+-----------+------+---------+-------+----------+-----+-----------+
|period|sectorId|sector-name|fuelId|fuel-name|stateId|state-name|value|value-units|
+------+--------+-----------+------+---------+-------+----------+-----+-----------+
|     0|       0|          0|     0|        0|      0|         0|    0|          0|
+------+--------+-----------+------+---------+-------+----------+-----+-----------+

Out[22]: 0

In [0]:
conditions_col1 = [
    (col("State_name").isin("Connecticut", "Maine", "Massachusetts", "New Hampshire", "New Jersey",
                        "New York", "Pennsylvania", "Rhode Island", "Vermont"), "Northeast"),
    (col("State_name").isin("Illinois", "Indiana", "Iowa", "Kansas", "Michigan", "Minnesota",
                        "Missouri", "Nebraska", "North Dakota", "Ohio", "South Dakota", "Wisconsin"), "Midwest"),
    (col("State_name").isin("Alabama", "Arkansas", "Delaware", "District of Columbia", "Florida", "Georgia",
                        "Kentucky", "Louisiana", "Maryland", "Mississippi", "North Carolina", "Oklahoma",
                        "South Carolina", "Tennessee", "Texas", "Virginia", "West Virginia"), "South"),
    (col("State_name").isin("Alaska", "Arizona", "California", "Colorado", "Hawaii", "Idaho", "Montana",
                        "Nevada", "New Mexico", "Oregon", "Utah", "Washington", "Wyoming"), "West")
]

result_df_co = df_co \
   .withColumn("regions", 
                when(conditions_col1[0][0], conditions_col1[0][1])
                .when(conditions_col1[1][0], conditions_col1[1][1])
                .when(conditions_col1[2][0], conditions_col1[2][1])
                .when(conditions_col1[3][0], conditions_col1[3][1])
                .otherwise("Other")) 

result_df_co.show()
result_df_co.count()

+------+--------+--------------------+------+-----------+-------+----------+---------+--------------------+-------+
|period|sectorId|              Sector|fuelId|       Fuel|stateId|State_name|    value|                unit|regions|
+------+--------+--------------------+------+-----------+-------+----------+---------+--------------------+-------+
|  2021|      RC|Residential carbo...|    CO|       Coal|     AK|    Alaska|      0.0|million metric to...|   West|
|  2021|      TT|Total carbon diox...|    NG|Natural Gas|     AK|    Alaska|19.623623|million metric to...|   West|
|  2021|      RC|Residential carbo...|    PE|  Petroleum|     AK|    Alaska| 0.616132|million metric to...|   West|
|  2021|      RC|Residential carbo...|    NG|Natural Gas|     AK|    Alaska| 1.113967|million metric to...|   West|
|  2021|      RC|Residential carbo...|    TO|  All Fuels|     AK|    Alaska| 1.730099|million metric to...|   West|
|  2021|      CC|Commercial carbon...|    CO|       Coal|     AK|    Ala

In [0]:
result_df_co.coalesce(1).write.mode("overwrite").option("header",'true').csv("/mnt/energyclimatenexus/Transformed_data/co2_emissions_data")